In [ ]:
import json
from tqdm import tqdm
import re
import torch

# 定义 color_mapping
color_mapping = {
    'red': [0.1, 0.15],
    'green': [0.15, 0.3],
    'blue': [0.3, 0.45],
    'yellow': [0.45, 0.6],
    'orange': [0.6, 0.75],
    'purple': [0.75, 0.9]
}

# 提取所有颜色
colors = color_mapping.keys()

# 检查颜色是否存在于字符串中
def find_colors_in_string(input_string):
    found_colors = [color for color in colors if color in input_string]
    return found_colors


def map_to_color(pixel):
    if pixel<0.1:
        return 'black'
    elif 0.1<=pixel<0.15:
        return 'red'
    elif 0.15<=pixel<0.3:
        return 'green'
    elif 0.3<=pixel<0.45:
        return 'blue'
    elif 0.45<=pixel<0.6:
        return 'yellow'
    elif 0.6<=pixel<0.75:
        return 'orange'
    elif 0.75<=pixel<=0.9:
        return 'purple'
    else:
        return 'other'

def compare_img(gen_img,gt_img):
    correct_pixel=0
    incorrect_pixel=0
    for i in range(len(gen_img)):
        for j in range(len(gen_img[i])):
            if map_to_color(gen_img[i][j])!=map_to_color(gt_img[i][j]):
                incorrect_pixel+=1
            else:
                correct_pixel+=1
    return correct_pixel,incorrect_pixel

acc=[]
answer_list=[i*45 for i in range(1,2)]
for answer in answer_list:
    #data_path=f'/datadrive_a/jihai/LLaVA/answer/answer-llava-v1.5-7b-mix-u-odd-{i}.jsonl'
    data_path=f'./answer/answer-llava-v1.5-7b-sw-lora-{answer}.jsonl'
    time_count=0
    time_score=0
    weather_count=0
    weather_score=0
    position_count=0
    position_score=0
    battery_count=0
    battery_score=0
    with open (data_path, "r") as f:
        for line in tqdm(f):
            json_obj = json.loads(line.strip())
            ground_truth=json_obj['groun_truth']
            answer=json_obj['answer']
            if ' ' not in ground_truth:
                if ':' in ground_truth:
                    time_count+=1
                    pattern = r"^(\d{2}):(\d{2}):(\d{2})$"
                    match = re.match(pattern, ground_truth)
                    gt_h = int(match.group(1))
                    gt_m = int(match.group(2))
                    gt_s = int(match.group(3))
                    match = re.match(pattern, answer)
                    if match:
                        ans_h = int(match.group(1))
                        ans_m = int(match.group(2))
                        ans_s = int(match.group(3))
                        err=(abs(ans_h - gt_h)/6.0 + abs(ans_m - gt_m)/30.0 + abs(ans_s - gt_s)/30.0)/3.0
                        time_score+=1-err
                elif 'sunny' in ground_truth or 'raining' in ground_truth or 'cloudy' in ground_truth:
                    weather_count+=1
                    if ground_truth in answer:
                        weather_score+=1
                elif '-' in ground_truth:
                    position_count+=1
                    if ground_truth in answer:
                        position_score+=1
                elif '%' in ground_truth:
                    battery_count+=1
                    match = re.search(r'\b(100|[1-9]\d?|0)%', ground_truth)
                    gt=int(match.group(1)) / 100
                    match = re.search(r'\b(100|[1-9]\d?|0)%', answer)
                    if match:
                        ans=int(match.group(1)) / 100
                        err=abs(ans - gt)
                        battery_score+=1-err
                else:
                    raise ValueError(f"Unknown ground truth format: {ground_truth}")
    time_acc=time_score/time_count
    weather_acc=weather_score/weather_count
    position_acc=position_score/position_count
    battery_acc=battery_score/battery_count
    total_acc=(time_score+weather_score+position_score+battery_score)/(time_count+weather_count+position_count+battery_count)
    acc.append([time_acc,weather_acc,position_acc,battery_acc,total_acc])
    print(f"time_acc: {time_acc:.4f}, weather_acc: {weather_acc:.4f}, position_acc: {position_acc:.4f}, battery_acc: {battery_acc:.4f}, total_acc: {total_acc:.4f}")
acc=torch.Tensor(acc)
acc=acc.permute(1,0)
print(f"time_acc:{acc[0]}")
print(f"weather_acc:{acc[1]}")
print(f"position_acc:{acc[2]}")
print(f"battery_acc:{acc[3]}")
print(f"total_acc:{acc[4]}")


200it [00:00, 10804.77it/s]


generation_accuracy:0.3333333333333333
time_accuracy:0.051587301587301584
color_accuracy:0.9523809523809523


200it [00:00, 5660.56it/s]


generation_accuracy:0.5242200328407225
time_accuracy:0.027777777777777776
color_accuracy:0.7261904761904762


200it [00:00, 10589.67it/s]


generation_accuracy:0.7830459770114943
time_accuracy:0.8333333333333334
color_accuracy:0.8690476190476191


200it [00:00, 5518.28it/s]


generation_accuracy:0.9113300492610837
time_accuracy:1.0
color_accuracy:0.9761904761904762


138it [00:00, 5153.49it/s]

generation_accuracy:0.9206349206349206
time_accuracy:1.0
color_accuracy:0.9473684210526315
tensor([0.3333, 0.5242, 0.7830, 0.9113, 0.9206])
tensor([0.0516, 0.0278, 0.8333, 1.0000, 1.0000])
tensor([0.9524, 0.7262, 0.8690, 0.9762, 0.9474])


In [2]:
import re
def parse_percent(s):
    match = re.search(r'\b(100|[1-9]\d?|0)%', s)
    if match:
        return int(match.group(1)) / 100
    return None

# 示例
print(parse_percent('60%'))    # 输出: 0.65
print(parse_percent('10%'))   # 输出: 1.0
print(parse_percent('0%'))     # 输出: 0.0

0.6
0.1
0.0


In [8]:
from sklearn import kernel_approximation
import torch
import torch.nn.functional as F
padded_input=torch.Tensor([[0., 0., 0., 1., 0., 0.],
        [1., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [1., 0., 1., 1., 1., 0.]]).unsqueeze(0).unsqueeze(0)
padded_input=torch.Tensor([[0., 1., 1., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 1., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0.]]).unsqueeze(0).unsqueeze(0)
kernel=torch.Tensor([[1, 0, 3],
    [0, 0, 2],
    [4, 1, 0]]).unsqueeze(0).unsqueeze(0) 
output = F.conv2d(padded_input, kernel, stride=1)
print(output)

tensor([[[[4., 9., 5., 4.],
          [2., 1., 4., 2.],
          [6., 6., 6., 7.],
          [5., 2., 1., 2.]]]])
